In [220]:
# Loading modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [221]:
# Loading data from Excel
df = pd.read_excel('data/LV_Datenbank.xlsx', sheet_name='Daten Trägersysteme 2021')
df_launchrates = pd.read_excel('data/LV_Datenbank_cleaned.xlsx', sheet_name='Startraten 2021')

In [222]:
# Creating a dataframe with all fuel types
df_fuel = pd.DataFrame(pd.unique(df[['stage1_fuel', 'stage2_fuel', 'stage3_fuel', 'stage4_fuel', 'stage5_fuel']].values.ravel('K')), columns=['fuel'])

# Dropping NaN Value
df_fuel = df_fuel.dropna()

# Setting Fuel as index
df_fuel = df_fuel.set_index('fuel')

In [223]:
# Adding the stoichiometric combustion products per kg fuel

# ['RP-1/LOX', 'Al/AP/HTPB', 'HTPB', 'N2O4/UDMH', 'LH2/LOX', 'Jet A/LOX', 'TH-H8299/Al', 'Hydrazine', 'MMH/MON']
df_fuel['CO2'] = np.array([3.15, 0.385690653, 3.214088774, 1.464583369, 0, 3.15, 0, 0, 0.955238504])
df_fuel['H2O'] = np.array([1.26, 0.277903767, 0.993343379, 1.199053817, 8.936682739, 1.26, 0, 1.124368235, 1.17308007])
df_fuel['N2'] = np.array([0, 0.0140067, 0, 1.398378524, 0, 0, 0, 1.311277585, 2.736174062])
df_fuel['O2'] = np.array([0, 0.23490668, 0, 0, 0, 0, 0, 0, 0])
df_fuel['HCl'] = np.array([0, 0.214130989, 0, 0, 0, 0, 0, 0, 0])
df_fuel['Al2O3'] = np.array([0, 0.358998351, 0, 0, 0, 0, 0, 0, 0])

print(df_fuel)

                  CO2       H2O        N2        O2       HCl     Al2O3
fuel                                                                   
RP-1/LOX     3.150000  1.260000  0.000000  0.000000  0.000000  0.000000
Al/AP/HTPB   0.385691  0.277904  0.014007  0.234907  0.214131  0.358998
HTPB         3.214089  0.993343  0.000000  0.000000  0.000000  0.000000
N2O4/UDMH    1.464583  1.199054  1.398379  0.000000  0.000000  0.000000
LH2/LOX      0.000000  8.936683  0.000000  0.000000  0.000000  0.000000
Jet A/LOX    3.150000  1.260000  0.000000  0.000000  0.000000  0.000000
TH-H8299/Al  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
Hydrazine    0.000000  1.124368  1.311278  0.000000  0.000000  0.000000
MMH/MON      0.955239  1.173080  2.736174  0.000000  0.000000  0.000000


In [224]:
# Replacing NaN with 0 within the original dataframe for further calculations
df = df.fillna(0)

In [225]:
# Creating a dataframe based on the factors of "df_fuel" and the used fuel type from "df"
# Multipling it by the stage mass for every product and every stage
for stage in np.arange(1, 6):
    for product in df_fuel.columns:
        df_temp = []
        for fuel_type in df['stage{0}_fuel'.format(stage)]:
            df_temp.append(df_fuel[product][fuel_type])
        df['stage{0}_{1}'.format(stage, product)] = df['stage{0}_mass_fuel'.format(stage)]*df_temp


In [226]:
# Creating the single sum (one launch) of every product for all launchvehicles

    # for product in df_fuel.columns:
    #     print(df[df.columns[df.columns.str.contains(product)]])
    
for product in df_fuel.columns:
    # Could result in false summation if the preceding products contain the name of the following ones
    df['single_sum_{0}'.format(product)] = df[df.columns[df.columns.str.contains('_{}'.format(product))]].sum(axis=1)


In [227]:
# Adding hand cleaned data of total launches from new Excel sheet

df["launchrate"] = df_launchrates["Total"]

In [228]:
# Creating the total sum (all launches) of every product for all launchvehicles

print("2021 - Total Emissions of Launchvehicles")
for product in df_fuel.columns:
    df['total_sum_{0}'.format(product)] = df['single_sum_{}'.format(product)] * df["launchrate"]
    
    print("{0}: {1} t".format(product, round(df['total_sum_{0}'.format(product)].sum()/1000, 2)))

2021 - Total Emissions of Launchvehicles
CO2: 44879.83 t
H2O: 23094.33 t
N2: 6323.11 t
O2: 169.13 t
HCl: 154.17 t
Al2O3: 258.48 t


In [229]:
# Exporting results

df.to_excel("output.xlsx") 
